In [3]:
! ls /kaggle/input/titanic

gender_submission.csv  test.csv  train.csv


In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
data_files = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        data_files.append(os.path.join(dirname, filename))
print(data_files)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

['/kaggle/input/titanic/train.csv', '/kaggle/input/titanic/test.csv', '/kaggle/input/titanic/gender_submission.csv']


In [5]:
data_files[0]

'/kaggle/input/titanic/train.csv'

In [7]:
df_train = pd.read_csv(data_files[0])
df_test = pd.read_csv(data_files[1])
df_gender = pd.read_csv(data_files[2])

print(df_train.shape, df_test.shape, df_gender.shape)

(891, 12) (418, 11) (418, 2)


In [12]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [13]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [15]:
df_train.Pclass.value_counts(normalize=True)

Pclass
3    0.551066
1    0.242424
2    0.206510
Name: proportion, dtype: float64

In [19]:
df_train.Sex.value_counts(normalize=True)

Sex
male      0.647587
female    0.352413
Name: proportion, dtype: float64

In [22]:
df_train[df_train.Age.isna()].sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
564,565,0,3,"Meanwell, Miss. (Marion Ogden)",female,NaN,0,0,SOTON/O.Q. 392087,8.0500,NaN,S
611,612,0,3,"Jardin, Mr. Jose Neto",male,NaN,0,0,SOTON/O.Q. 3101305,7.0500,NaN,S
330,331,1,3,"McCoy, Miss. Agnes",female,NaN,2,0,367226,23.2500,NaN,Q
475,476,0,1,"Clifford, Mr. George Quincy",male,NaN,0,0,110465,52.0000,A14,S
568,569,0,3,"Doharr, Mr. Tannous",male,NaN,0,0,2686,7.2292,NaN,C
667,668,0,3,"Rommetvedt, Mr. Knud Paust",male,NaN,0,0,312993,7.7750,NaN,S
101,102,0,3,"Petroff, Mr. Pastcho (""Pentcho"")",male,NaN,0,0,349215,7.8958,NaN,S
457,458,1,1,"Kenyon, Mrs. Frederick R (Marion)",female,NaN,1,0,17464,51.8625,D21,S
517,518,0,3,"Ryan, Mr. Patrick",male,NaN,0,0,371110,24.1500,NaN,Q
168,169,0,1,"Baumann, Mr. John D",male,NaN,0,0,PC 17318,25.9250,NaN,S


# Feature Engineering

## Title

In [44]:
df_train['Name'].apply(lambda x : x.split('.')[0].split(' ')[-1].strip()).value_counts()

Name
Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Mlle          2
Major         2
Col           2
Countess      1
Capt          1
Ms            1
Sir           1
Lady          1
Mme           1
Don           1
Jonkheer      1
Name: count, dtype: int64

In [45]:
df_train['Title'] = df_train['Name'].apply(lambda x : x.split('.')[0].split(' ')[-1].strip())

df_train.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
352,353,0,3,"Elias, Mr. Tannous",male,15.00,1,1,2695,7.2292,NaN,C,Mr
804,805,1,3,"Hedman, Mr. Oskar Arvid",male,27.00,0,0,347089,6.9750,NaN,S,Mr
469,470,1,3,"Baclini, Miss. Helene Barbara",female,0.75,2,1,2666,19.2583,NaN,C,Miss
440,441,1,2,"Hart, Mrs. Benjamin (Esther Ada Bloomfield)",female,45.00,1,1,F.C.C. 13529,26.2500,NaN,S,Mrs
830,831,1,3,"Yasbeck, Mrs. Antoni (Selini Alexander)",female,15.00,1,0,2659,14.4542,NaN,C,Mrs
47,48,1,3,"O'Driscoll, Miss. Bridget",female,NaN,0,0,14311,7.7500,NaN,Q,Miss
327,328,1,2,"Ball, Mrs. (Ada E Hall)",female,36.00,0,0,28551,13.0000,D,S,Mrs
64,65,0,1,"Stewart, Mr. Albert A",male,NaN,0,0,PC 17605,27.7208,NaN,C,Mr
738,739,0,3,"Ivanoff, Mr. Kanio",male,NaN,0,0,349201,7.8958,NaN,S,Mr
776,777,0,3,"Tobin, Mr. Roger",male,NaN,0,0,383121,7.7500,F38,Q,Mr


In [46]:
df_train.Age.mean()

29.69911764705882

In [53]:
df_train.groupby("Title").agg({'Age':['count', 'mean']}).sort_values(('Age', 'mean'), ascending = False)

Age           
         count       mean
Title                    
Capt         1  70.000000
Col          2  58.000000
Sir          1  49.000000
Major        2  48.500000
Lady         1  48.000000
Rev          6  43.166667
Dr           6  42.000000
Don          1  40.000000
Jonkheer     1  38.000000
Mrs        108  35.898148
Countess     1  33.000000
Mr         398  32.368090
Ms           1  28.000000
Mlle         2  24.000000
Mme          1  24.000000
Miss       146  21.773973
Master      36   4.574167

In [57]:
df_train[df_train.Age.isna()].shape

(177, 13)

In [58]:
df_train[df_train.Age.isna()].Title.value_counts()

Title
Mr        119
Miss       36
Mrs        17
Master      4
Dr          1
Name: count, dtype: int64

**Note** Use the average age by `Title` to encode missing value in the `Age` column. This method is preferred over mean encoding on the overall dataset.

In [71]:
df_title_mean_age = df_train.groupby("Title").agg({'Age':'mean'}).reset_index().rename({'Age': 'Title_Mean_Age'}, axis = 1)

df_title_mean_age.sample(5)

,Title,Title_Mean_Age
3,Don,40.000000
8,Master,4.574167
7,Major,48.500000
5,Jonkheer,38.000000
11,Mme,24.000000


In [75]:
df_train = df_train.merge(df_title_mean_age, on = 'Title')

In [76]:
df_train[df_train.Age.isna()].sample()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Title_Mean_Age
359,360,1,3,"Mockler, Miss. Helen Mary ""Ellie""",female,NaN,0,0,330980,7.8792,NaN,Q,Miss,21.773973


In [79]:
df_train.loc[df_train.Age.isna(), 'Age'] = df_train['Title_Mean_Age']

df_train.iloc[359]

PassengerId                                     360
Survived                                          1
Pclass                                            3
Name              Mockler, Miss. Helen Mary "Ellie"
Sex                                          female
Age                                       21.773973
SibSp                                             0
Parch                                             0
Ticket                                       330980
Fare                                         7.8792
Cabin                                           NaN
Embarked                                          Q
Title                                          Miss
Title_Mean_Age                            21.773973
Name: 359, dtype: object

In [80]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PassengerId     891 non-null    int64  
 1   Survived        891 non-null    int64  
 2   Pclass          891 non-null    int64  
 3   Name            891 non-null    object 
 4   Sex             891 non-null    object 
 5   Age             891 non-null    float64
 6   SibSp           891 non-null    int64  
 7   Parch           891 non-null    int64  
 8   Ticket          891 non-null    object 
 9   Fare            891 non-null    float64
 10  Cabin           204 non-null    object 
 11  Embarked        889 non-null    object 
 12  Title           891 non-null    object 
 13  Title_Mean_Age  891 non-null    float64
dtypes: float64(3), int64(5), object(6)
memory usage: 97.6+ KB


In [90]:
df_train.Embarked.value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [91]:
df_train[['Embarked','Pclass', 'Ticket']].sample(10)

,Embarked,Pclass,Ticket
751,S,3,392096
805,S,3,347063
845,S,3,C.A. 5547
327,S,2,28551
145,S,2,C.A. 33112
420,C,3,349254
121,S,3,A4. 54510
656,S,3,349223
180,S,3,CA. 2343
431,S,3,376564


In [92]:
df_train.Fare.sample(10)

213     13.0000
319    134.5000
801     26.2500
19       7.2250
603      8.0500
209     31.0000
528      7.9250
698    110.8833
228     13.0000
568      7.2292
Name: Fare, dtype: float64

In [94]:
df_train.groupby(['Pclass', 'Embarked']).agg({'Fare':'mean'})

Fare
Pclass Embarked            
1      C         104.718529
       Q          90.000000
       S          70.364862
2      C          25.358335
       Q          12.350000
       S          20.327439
3      C          11.214083
       Q          11.183393
       S          14.644083

In [102]:
df_train.dtypes

PassengerId         int64
Survived            int64
Pclass              int64
Name               object
Sex                object
Age               float64
SibSp               int64
Parch               int64
Ticket             object
Fare              float64
Cabin              object
Embarked           object
Title              object
Title_Mean_Age    float64
dtype: object

In [104]:
df_train[df_train.Cabin.notnull()].groupby('Pclass')['Cabin'].agg(' '.join)

Pclass
1    C85 C123 E46 C103 A6 C23 C25 C27 B78 D33 B30 C...
2    D56 F33 E101 F2 F4 F2 D E101 D F2 F33 D F33 F4...
3    G6 F G73 F E69 G6 G6 G6 E10 F G63 F G73 E121 F...
Name: Cabin, dtype: object

In [108]:
class_survivors = df_train.groupby('Pclass').agg({'Survived':['count', 'sum']})

class_survivors

Survived     
          count  sum
Pclass              
1           216  136
2           184   87
3           491  119

In [111]:
class_survivors[('Survived', 'sum')]/class_survivors[('Survived', 'count')]

Pclass
1    0.629630
2    0.472826
3    0.242363
dtype: float64